In [ ]:
import functools
import numpy  as np
from IPython.core.debugger import set_trace

https://github.com/karpathy/micrograd

In [ ]:
# make base class

class Tensor:
    def __init__(self, data):
        if type(data) != np.ndarray:
            print('error type data %r' % data)
            assert(False)
        self.data = data
        self.grad = None
        self._ctx = None

    def backward(self, fill = True):
        
        if self._ctx is None:
            return

        if self.grad is None and fill:
            # iniy first grad ones
            assert self.data.size == 1
            self.grad = np.ones_like(self.data)
        
        assert(self.grad is not None)  

        """
        _ctx.backward return from <class '__main__.Mul'> 
        and method Mul.backward return y * grad_out, x * grad_out
        
        """      
        grads = self._ctx.backward(self._ctx, self.grad)
        
        if len(self._ctx.parents) == 1:
            grads = [grads]
        for t, g in zip(self._ctx.parents, grads):
            # t  Tensor
            if g.shape != t.data.shape:
                print('grad shape must match tensor shape')
                assert(False)           
            # add grad to Tensor
            t.grad = g
            t.backward(False)

"""
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html#:~:text=beginner%2Fexamples_autograd%2Ftwo_layer_net_custom_function-,PyTorch%3A%20Defining%20New%20autograd%20Functions,PyTorch%20autograd%20to%20compute%20gradients.
В прямом проходе мы получаем тензор, содержащий ввод и возврат
Тензор, содержащий вывод. 

ctx - это объект контекста, который можно использовать
хранить информацию для обратных вычислений. Вы можете кешировать произвольные
объекты для использования в обратном проходе с помощью метода ctx.save_for_backward.


При обратном проходе мы получаем тензор, содержащий градиент потери
относительно выхода, и нам нужно вычислить градиент потерь относительно входа.

"""

class Function:

    def __init__(self, *tensor):
        self.parents = tensor
        self.saved_tensors = []
    
    def save_for_backward(self, *x):
        self.saved_tensors.extend(x)    

    def apply(self, arg, *x):
        """
        a = Tensor(np.array([1, 2]))
        b = Tensor(np.array([3, 2]))
        
        a.mul(b)
        
        arg: <class '__main__.Mul'> 
        self.data -> Tensor.data
        
        """
        
        ctx = arg(self, *x)
        ret = Tensor(arg.forward(ctx, self.data, *[t.data for t in x]))
        ret._ctx = ctx
        return ret

def register(name, fn):
    setattr(Tensor, name, functools.partialmethod(fn.apply, fn))

class Mul(Function):

    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x * y

    @staticmethod
    def backward(ctx, grad_out):       
        # set_trace()
        x, y = ctx.saved_tensors       
        return y * grad_out, x * grad_out

register('mul', Mul)

In [ ]:
a = Tensor(np.array([1]))
b = Tensor(np.array([3]))
c = a.mul(b)


In [ ]:
c.backward()

<__main__.Tensor object at 0x7fbfbe2e7c88> [3] zip
<__main__.Tensor object at 0x7fbfbe2e74e0> [1] zip


In [ ]:
a.grad

array([3])